# Examen final – Modelos no lineales para pronóstico

## Pronóstico de carga viral de SARS-CoV-2 en aguas residuales

> Notebook plantilla listo para empezar. La idea es que rellenes los `TODO` con tu propio desarrollo, sin tener que pelearte con la estructura básica.

### 1. Contexto general

Desde la pandemia de COVID-19, la **vigilancia basada en aguas residuales** (wastewater-based epidemiology) se usa para monitorear virus respiratorios a nivel poblacional. En lugar de depender solo de pruebas diagnósticas individuales, se analizan muestras de plantas de tratamiento para cuantificar la cantidad de **ARN viral** presente, usando técnicas como RT-qPCR o RT-dPCR.

El **National Wastewater Surveillance System (NWSS)** del CDC (Centers for Disease Control and Prevention) publica datos abiertos con la evolución de la carga viral de SARS-CoV-2 en aguas residuales para múltiples sitios y estados, actualizados de forma periódica. Estas series permiten detectar cambios en la circulación del virus incluso **varios días antes** que los cambios en casos clínicos, y capturan también infecciones asintomáticas.

### 2. Problema a resolver

En este proyecto se trabajará con una **serie de tiempo univariada** que representa la evolución de la carga viral de SARS-CoV-2 en aguas residuales para un sitio / región específica (por ejemplo, una planta de tratamiento o un estado). Para cada fecha de muestreo se tiene una medida de concentración de ARN viral normalizada o una métrica derivada.

Objetivo principal:

> **Construir y evaluar modelos de series de tiempo para pronosticar la carga viral de SARS-CoV-2 en aguas residuales en las próximas N semanas** a partir del histórico disponible.

En términos prácticos:

- La variable objetivo será una medida numérica asociada a la **carga viral** en aguas residuales (por ejemplo, una concentración normalizada o índice de actividad viral).
- Trabajaremos con granularidad temporal según el dataset (diaria o semanal).
- Usaremos ventanas deslizantes del tipo:
  
  $[
  [y_{t-w+1}, \dots, y_t] \rightarrow y_{t+1}
  $]

### 3. Motivación

1. **Salud pública y decisiones**: un buen pronóstico de la carga viral en aguas residuales permite anticipar aumentos de circulación antes de saturar hospitales o ver picos en los casos reportados.
2. **Componente bio / genético real**: aunque el análisis final es una serie de tiempo agregada, el dato de origen viene de cuantificar **ARN viral** con técnicas moleculares. Es un ejemplo directo de cómo la biotecnología y la genómica se conectan con modelos predictivos.
3. **Relevancia actual**: sistemas como NWSS siguen activos y se expanden a nuevos patógenos (influenza, RSV, mpox, etc.), por lo que el enfoque es reutilizable.

### 4. TODO: personaliza el problema con tus palabras

- [ ] Describe en 2–3 párrafos por qué este tema te interesa (bio, datos, salud pública, etc.).
- [ ] Define tu horizonte de predicción: ¿N días? ¿N semanas? (p. ej. 14 o 28 días).
- [ ] Explica qué tipo de decisiones o insights se podrían obtener a partir del pronóstico.


## Checklist del examen (mini guía)

Esta sección resume lo que debes cubrir según la rúbrica, para que no se te pase nada:

1. **Problema y datos**
   - Explicar variable, contexto y objetivo del pronóstico.
   - Describir bien la fuente de datos (CDC / NWSS) y el proceso de descarga automática.
   - Hacer exploración inicial: gráficos, tendencia, posibles estacionalidades.

2. **Preparación del dataset**
   - Crear ventanas deslizantes sin fuga de información.
   - Justificar cualquier transformación (log, escalamiento, etc.).
   - Documentar cómo se hace el split en train / val / test.

3. **Modelado**
   - Baseline ingenuo (último valor) como referencia.
   - Al menos un modelo no lineal (LSTM, CNN 1D, MLP, XGBoost, etc.).
   - Describir arquitectura e hiperparámetros y por qué se eligieron.

4. **Evaluación y resultados**
   - Usar mínimo 3 métricas (MAE, RMSE, MAPE/sMAPE, etc.).
   - Graficar real vs predicho en train/val/test o al menos en test.

5. **Pronóstico y conclusiones**
   - Mostrar pronóstico futuro (tabla + gráfica) y discutir si es coherente.
   - Opcional: predicciones para fechas específicas.
   - Redactar conclusiones claras, incluyendo limitaciones y mejoras futuras.


## Importación de librerías, configuración de directorios y seeds globales


In [2]:
import os
import sys
import random
from pathlib import Path
from typing import Dict, Tuple, List, Optional, Iterable

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go
import requests

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Opcional: descomposición, ACF/PACF
# import statsmodels.api as sm
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Opcional: deep learning (elige 1 stack y descomenta)
# import tensorflow as tf
# from tensorflow import keras
# OR
# import torch
# from torch import nn

# Configuración de gráficos
# plt.style.use("seaborn-v0_8")
# sns.set_palette("muted")

# Directorios base
PROJECT_ROOT = Path(".").resolve()
DATA_DIR = PROJECT_ROOT / "data"
LANDING_DIR = DATA_DIR / "landing"   # en vez de 'raw'
PROCESSED_DIR = DATA_DIR / "processed"
FIGURES_DIR = PROJECT_ROOT / "figures"
MODELS_DIR = PROJECT_ROOT / "models"

for d in (DATA_DIR, LANDING_DIR, PROCESSED_DIR, FIGURES_DIR, MODELS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Seed global
GLOBAL_SEED = 63

def set_global_seed(seed: int = 63):
    """Fija la seed para numpy, random y opcionalmente TF / Torch."""
    global GLOBAL_SEED
    GLOBAL_SEED = seed
    random.seed(seed)
    np.random.seed(seed)
    
    # try:
    #     tf.random.set_seed(seed)
    # except Exception:
    #     pass
    # 
    # try:
    #     torch.manual_seed(seed)
    #     if torch.cuda.is_available():
    #         torch.cuda.manual_seed_all(seed)
    # except Exception:
    #     pass

set_global_seed(GLOBAL_SEED)

print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Seed global fijada en {GLOBAL_SEED}")
print(f"LANDING_DIR: {LANDING_DIR}")
print(f"PROCESSED_DIR: {PROCESSED_DIR}")


PROJECT_ROOT: C:\Users\esteb\apps\Wastewater-SARS-CoV-2
Seed global fijada en 63
LANDING_DIR: C:\Users\esteb\apps\Wastewater-SARS-CoV-2\data\landing
PROCESSED_DIR: C:\Users\esteb\apps\Wastewater-SARS-CoV-2\data\processed


## Helpers generales: ventanas, métricas, gráficas y pronósticos


In [3]:
def create_sliding_windows(
    series: np.ndarray,
    window_size: int,
    horizon: int = 1,
    stride: int = 1
) -> Tuple[np.ndarray, np.ndarray]:
    """Crea ventanas deslizantes sin fuga de información."""
    series = np.asarray(series).astype(float)
    T = len(series)
    if T < window_size + horizon:
        raise ValueError("Serie demasiado corta para el window_size y horizon dados.")

    X, y = [], []
    last_start = T - window_size - horizon + 1
    for start in range(0, last_start, stride):
        end = start + window_size
        target_end = end + horizon
        X.append(series[start:end])
        y.append(series[end:target_end])

    X = np.stack(X)
    y = np.stack(y)
    return X, y


def time_series_train_val_test_split(
    X: np.ndarray,
    y: np.ndarray,
    train_frac: float = 0.7,
    val_frac: float = 0.15
) -> Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]:
    """Divide en train / val / test respetando el orden temporal."""
    n = len(X)
    n_train = int(n * train_frac)
    n_val = int(n * val_frac)
    n_test = n - n_train - n_val
    if n_test <= 0:
        raise ValueError("Demasiados pocos datos para este split. Ajusta train_frac/val_frac.")

    X_train, y_train = X[:n_train], y[:n_train]
    X_val, y_val = X[n_train:n_train + n_val], y[n_train:n_train + n_val]
    X_test, y_test = X[n_train + n_val:], y[n_train + n_val:]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """Calcula MAE, RMSE y MAPE (maneja ceros con un pequeño epsilon)."""
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    eps = 1e-8
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), eps))) * 100

    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}


def plot_time_series(dates, values, title: str = "", ylabel: str = ""):
    """Gráfica una serie de tiempo usando Plotly."""
    dates = pd.to_datetime(dates)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=values,
            mode="lines+markers",
            name="Serie"
        )
    )
    fig.update_layout(
        title=title,
        xaxis_title="Fecha",
        yaxis_title=ylabel,
        template="plotly_white"
    )
    return fig


def plot_history_vs_pred(
    dates,
    y_true: np.ndarray,
    y_pred: np.ndarray,
    title: str = "Histórico vs predicción"
):
    """Gráfica histórico vs predicho usando Plotly."""
    dates = pd.to_datetime(dates)
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=y_true,
            mode="lines",
            name="Real"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=y_pred,
            mode="lines",
            name="Predicho",
            line=dict(dash="dash")
        )
    )
    fig.update_layout(
        title=title,
        xaxis_title="Fecha",
        yaxis_title="Valor",
        template="plotly_white"
    )
    return fig


def plot_test_vs_pred(
    dates_test,
    y_test: np.ndarray,
    y_pred_test: np.ndarray,
    title: str = "Test vs predicción"
):
    """Wrapper para test vs predicción."""
    return plot_history_vs_pred(dates_test, y_test, y_pred_test, title=title)


def plot_future_forecast(
    history_dates,
    history_values,
    future_dates,
    future_preds,
    title: str = "Pronóstico futuro"
):
    """Gráfica histórico + pronóstico futuro con Plotly."""
    history_dates = pd.to_datetime(history_dates)
    future_dates = pd.to_datetime(future_dates)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=history_dates,
            y=history_values,
            mode="lines",
            name="Histórico"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=future_dates,
            y=future_preds,
            mode="lines+markers",
            name="Pronóstico",
            line=dict(dash="dash")
        )
    )
    fig.update_layout(
        title=title,
        xaxis_title="Fecha",
        yaxis_title="Valor",
        template="plotly_white"
    )
    return fig


def default_predict_fn(model, X):
    """Wrapper genérico para .predict o llamado directo."""
    if hasattr(model, "predict"):
        return model.predict(X)
    else:
        return model(X)


def recursive_forecast(
    model,
    last_window: np.ndarray,
    n_future: int,
    predict_fn=default_predict_fn
) -> np.ndarray:
    """Pronóstico recursivo a n_future pasos usando un modelo one-step-ahead."""
    window = np.asarray(last_window).reshape(1, -1)
    preds = []
    for _ in range(n_future):
        y_hat = predict_fn(model, window)
        y_hat = np.asarray(y_hat).reshape(-1)
        y_next = float(y_hat[0])
        preds.append(y_next)
        # Deslizar ventana
        window = np.roll(window, -1, axis=1)
        window[0, -1] = y_next
    return np.array(preds)


def forecast_for_specific_dates(
    model,
    series: pd.Series,
    future_dates: List,
    window_size: int,
    predict_fn=default_predict_fn
) -> pd.Series:
    """Pronostica valores para fechas específicas futuras, asumiendo serie con índice datetime."""
    series = series.sort_index()
    last_date = series.index[-1]
    last_window = series.values[-window_size:]

    future_dates = pd.to_datetime(pd.Index(future_dates))
    future_dates = future_dates.sort_values()

    steps_ahead = ((future_dates - last_date) / np.timedelta64(1, "D")).astype(int)
    max_steps = steps_ahead.max()
    if max_steps <= 0:
        raise ValueError("Todas las fechas específicas deben ser posteriores al último dato.")

    all_future = recursive_forecast(model, last_window, int(max_steps), predict_fn=predict_fn)

    values = []
    for s in steps_ahead:
        if s <= 0:
            values.append(np.nan)
        else:
            values.append(all_future[int(s) - 1])

    return pd.Series(values, index=future_dates, name=series.name)


class NaiveLastValueModel:
    """Baseline ingenuo: pronostica el último valor de la ventana."""
    def __init__(self):
        self.fitted_ = False

    def fit(self, X, y=None):
        self.fitted_ = True
        return self

    def predict(self, X):
        X = np.asarray(X)
        last_vals = X[:, -1]
        return last_vals.reshape(-1, 1)


print("Helpers cargados (versión Plotly): create_sliding_windows, regression_metrics, plot_*, recursive_forecast, forecast_for_specific_dates, NaiveLastValueModel.")


Helpers cargados (versión Plotly): create_sliding_windows, regression_metrics, plot_*, recursive_forecast, forecast_for_specific_dates, NaiveLastValueModel.


## Datos NWSS: configuración de endpoints y descarga automática (landing)

Vamos a usar el dataset público del CDC:

- **ID:** `j9g8-acpt`
- **Nombre:** CDC Wastewater Data for SARS-CoV-2
- **Fuente:** data.cdc.gov


In [4]:
# Endpoints oficiales del CDC para aguas residuales (SARS-CoV-2)

CDC_WASTEWATER_CSV_URL = (
    "https://data.cdc.gov/api/views/j9g8-acpt/rows.csv?accessType=DOWNLOAD"
)

# Endpoint API tipo SODA para consultas filtradas
CDC_WASTEWATER_API_BASE = "https://data.cdc.gov/resource/j9g8-acpt.csv"

# Archivo local en la capa landing
CDC_WASTEWATER_FULL_CSV_PATH = LANDING_DIR / "cdc_wastewater_sarscov2_full.csv"

print("CDC_WASTEWATER_CSV_URL:", CDC_WASTEWATER_CSV_URL)
print("CDC_WASTEWATER_API_BASE:", CDC_WASTEWATER_API_BASE)
print("CDC_WASTEWATER_FULL_CSV_PATH:", CDC_WASTEWATER_FULL_CSV_PATH)


CDC_WASTEWATER_CSV_URL: https://data.cdc.gov/api/views/j9g8-acpt/rows.csv?accessType=DOWNLOAD
CDC_WASTEWATER_API_BASE: https://data.cdc.gov/resource/j9g8-acpt.csv
CDC_WASTEWATER_FULL_CSV_PATH: C:\Users\esteb\apps\Wastewater-SARS-CoV-2\data\landing\cdc_wastewater_sarscov2_full.csv


In [5]:
def download_cdc_wastewater_full(
    url: str = CDC_WASTEWATER_CSV_URL,
    out_path: Path = CDC_WASTEWATER_FULL_CSV_PATH,
    use_cache: bool = True,
    chunk_size: int = 1_000_000,
) -> Path:
    """Descarga el CSV completo de 'CDC Wastewater Data for SARS-CoV-2' a la capa landing.

    Si use_cache=True y el archivo ya existe, no se vuelve a descargar.
    """
    out_path = Path(out_path)

    if use_cache and out_path.exists():
        print(f"[INFO] Usando archivo local en caché: {out_path}")
        return out_path

    print(f"[INFO] Descargando datos desde:\n  {url}")
    try:
        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
    except requests.RequestException as e:
        print("[ERROR] Falló la descarga desde CDC.")
        print("Detalle:", e)
        raise

    print(f"[OK] Datos guardados en: {out_path}")
    return out_path


def load_cdc_wastewater_full(
    csv_path: Path = CDC_WASTEWATER_FULL_CSV_PATH,
    parse_dates: bool = True,
) -> pd.DataFrame:
    """Carga el CSV completo de aguas residuales en un DataFrame.

    NOTA: el dataset es grande; si solo necesitas un sitio, quizá convenga usar
    la función de API filtrada en lugar de todo el CSV.
    """
    csv_path = Path(csv_path)

    if not csv_path.exists():
        raise FileNotFoundError(
            f"No se encontró el archivo {csv_path}. "
            "Ejecuta primero download_cdc_wastewater_full()."
        )

    print(f"[INFO] Cargando datos desde {csv_path} ...")
    df = pd.read_csv(csv_path, low_memory=False)

    print(f"[INFO] DataFrame cargado con shape: {df.shape}")

    if parse_dates:
        # TODO: ajusta el nombre real de la columna de fecha según df.columns
        date_candidates = [
            "sample_collect_date",
            "collection_date",
            "sample_date",
        ]
        for col in date_candidates:
            if col in df.columns:
                print(f"[INFO] Parseando columna de fecha: {col}")
                df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)
                break
        else:
            print(
                "[WARN] No se reconoció automáticamente la columna de fecha. "
                "Revisa df.columns y actualiza este bloque."
            )

    return df


def fetch_cdc_wastewater_subset(
    jurisdiction: Optional[str] = None,
    sewershed_ids: Optional[Iterable[str]] = None,
    min_date: Optional[str] = None,
    max_date: Optional[str] = None,
    limit: int = 50_000,
    app_token: Optional[str] = None,
) -> pd.DataFrame:
    """Trae un subconjunto de datos desde la API SODA del CDC.

    Parameters
    ----------
    jurisdiction : str, optional
        Código de jurisdicción (ej. estado tipo 'CA', 'TX', etc.).
    sewershed_ids : iterable of str, optional
        Lista de IDs de sewershed a filtrar.
    min_date, max_date : str, optional
        Fechas 'YYYY-MM-DD' para filtrar por rango de colección.
    limit : int
        Límite de filas por llamada.
    app_token : str, optional
        Token de app de data.cdc.gov (opcional pero útil si haces muchas consultas).
    """
    params = {
        "$limit": limit,
    }

    where_clauses = []

    # TODO: ajusta nombres de columnas una vez veas df_raw.columns
    date_col = "sample_collect_date"   # cambiar si el nombre es otro
    jurisdiction_col = "wwtp_jurisdiction"  # ejemplo
    sewershed_col = "sewershed_id"         # ejemplo

    if jurisdiction:
        where_clauses.append(
            f"upper({jurisdiction_col}) = '{jurisdiction.upper()}'"
        )

    if sewershed_ids:
        ids_str = ",".join(f"'{sid}'" for sid in sewershed_ids)
        where_clauses.append(f"{sewershed_col} in ({ids_str})")

    if min_date:
        where_clauses.append(f"{date_col} >= '{min_date}'")
    if max_date:
        where_clauses.append(f"{date_col} <= '{max_date}'")

    if where_clauses:
        params["$where"] = " AND ".join(where_clauses)

    print("[INFO] Llamando a la API SODA de CDC con params:")
    for k, v in params.items():
        print(f"  {k}: {v}")

    headers = {}
    if app_token:
        headers["X-App-Token"] = app_token

    try:
        r = requests.get(CDC_WASTEWATER_API_BASE, params=params, headers=headers, timeout=60)
        r.raise_for_status()
        from io import StringIO
        df = pd.read_csv(StringIO(r.text))
    except Exception as e:
        print("[ERROR] No se pudo leer desde la API SODA.")
        print("Detalle:", e)
        raise

    print(f"[OK] Subconjunto recibido con shape: {df.shape}")
    return df


print("Funciones de descarga/carga CDC listas: download_cdc_wastewater_full, load_cdc_wastewater_full, fetch_cdc_wastewater_subset.")


Funciones de descarga/carga CDC listas: download_cdc_wastewater_full, load_cdc_wastewater_full, fetch_cdc_wastewater_subset.


## Selección de sewershed / región y construcción de la serie de tiempo

Aquí vas a elegir **qué sitio / región específica** vas a modelar.

Checklist:

- [ ] Decidir si trabajarás con un **estado completo** o con un **sewershed específico**.
- [ ] Definir rango de fechas (por ejemplo, de 2021-01-01 a la fecha más reciente disponible).
- [ ] Seleccionar la columna que usarás como **fecha** y la que usarás como **target** (por ejemplo una concentración normalizada o índice de actividad viral).


In [6]:
# TODO: elige una de estas estrategias para obtener los datos de tu sitio/región

# Opción A: usar el CSV completo (landing) y luego filtrar
# download_cdc_wastewater_full(use_cache=True)
df_raw = load_cdc_wastewater_full()

# Opción B: usar solo un subconjunto vía API (recomendable si ya tienes claro el sitio)
# df_raw = fetch_cdc_wastewater_subset(
#     jurisdiction="CA",        # TODO: cambia por el estado/jurisdicción
#     sewershed_ids=None,        # TODO: puedes pasar una lista de IDs concretos
#     min_date="2021-01-01",    # TODO: ajusta rango de fechas
#     max_date=None,             # None = hasta la fecha más reciente
#     limit=50000,
#     app_token=None,            # Si tienes un app token, lo puedes poner aquí
# )

# TODO: descomenta UNA de las dos opciones de arriba y explora df_raw
df_raw.head()
df_raw.columns


[INFO] Cargando datos desde C:\Users\esteb\apps\Wastewater-SARS-CoV-2\data\landing\cdc_wastewater_sarscov2_full.csv ...
[INFO] DataFrame cargado con shape: (504369, 35)
[INFO] Parseando columna de fecha: sample_collect_date


Index(['sewershed_id', 'wwtp_jurisdiction', 'county_fips', 'counties_served',
       'population_served', 'sample_id', 'sample_collect_date', 'sample_type',
       'sample_matrix', 'sample_location', 'flow_rate', 'concentration_method',
       'pasteurized', 'pcr_type', 'extraction_method', 'major_lab_method',
       'inhibition_detect', 'inhibition_adjust', 'ntc_amplify', 'pcr_target',
       'pcr_gene_target_agg', 'pcr_target_avg_conc', 'pcr_target_units',
       'lod_sewage', 'pcr_target_avg_conc_lin', 'pcr_target_flowpop_lin',
       'pcr_target_mic_lin', 'hum_frac_target_mic', 'hum_frac_mic_conc',
       'hum_frac_mic_unit', 'rec_eff_percent', 'rec_eff_target_name',
       'rec_eff_spike_matrix', 'rec_eff_spike_conc', 'date_updated'],
      dtype='object')

In [7]:
display(df_raw.head())
print(df_raw.columns)

,sewershed_id,wwtp_jurisdiction,county_fips,counties_served,population_served,sample_id,sample_collect_date,sample_type,sample_matrix,sample_location,...,pcr_target_flowpop_lin,pcr_target_mic_lin,hum_frac_target_mic,hum_frac_mic_conc,hum_frac_mic_unit,rec_eff_percent,rec_eff_target_name,rec_eff_spike_matrix,rec_eff_spike_conc,date_updated
0,711,me,23019,Penobscot,2500,000516f8c0f05102d4a010b987f62273,2023-11-28 00:00:00+00:00,24-hr flow-weighted composite,raw wastewater,wwtp,...,3.634599e+08,0.02864,pepper mild mottle virus,1.002597e+07,copies/l wastewater,68.74511,brsv vaccine,raw sample post pasteurization,5.34357,09/26/2025 10:40:00 AM
1,1809,ri,44007,Providence,10000,040ce1a855db659d046911c5d5758314,2023-07-05 00:00:00+00:00,24-hr time-weighted composite,raw wastewater,wwtp,...,4.267105e+07,0.00092,pepper mild mottle virus,1.772809e+08,copies/l wastewater,80.15343,brsv vaccine,raw sample post pasteurization,5.00000,09/26/2025 10:40:00 AM
2,322,fl,12115,Sarasota,100000,052760ee8f2bec3e7e4ac25f5bff23b4,2023-08-14 00:00:00+00:00,24-hr flow-weighted composite,post grit removal,wwtp,...,1.263573e+08,0.00455,pepper mild mottle virus,1.163784e+08,copies/l wastewater,20.63618,brsv vaccine,raw sample post pasteurization,5.34357,09/26/2025 10:40:00 AM
3,524,in,18113,Noble,10000,0dd046c819c8214f02eb79056e57978a,2023-12-18 00:00:00+00:00,24-hr time-weighted composite,raw wastewater,wwtp,...,5.170796e+07,0.00177,pmmov (gt-digital),4.752000e+07,copies/l wastewater,33.02887,bcov vaccine,raw sample,5.45000,09/26/2025 10:40:00 AM
4,694,me,23001,Androscoggin,60000,0e08cd627f3702430558aaf38aefa6e4,2023-09-13 00:00:00+00:00,24-hr flow-weighted composite,raw wastewater,wwtp,...,1.066697e+08,0.01578,pepper mild mottle virus,1.410123e+07,copies/l wastewater,6.26729,brsv vaccine,raw sample post pasteurization,5.34357,09/26/2025 10:40:00 AM


Index(['sewershed_id', 'wwtp_jurisdiction', 'county_fips', 'counties_served',
       'population_served', 'sample_id', 'sample_collect_date', 'sample_type',
       'sample_matrix', 'sample_location', 'flow_rate', 'concentration_method',
       'pasteurized', 'pcr_type', 'extraction_method', 'major_lab_method',
       'inhibition_detect', 'inhibition_adjust', 'ntc_amplify', 'pcr_target',
       'pcr_gene_target_agg', 'pcr_target_avg_conc', 'pcr_target_units',
       'lod_sewage', 'pcr_target_avg_conc_lin', 'pcr_target_flowpop_lin',
       'pcr_target_mic_lin', 'hum_frac_target_mic', 'hum_frac_mic_conc',
       'hum_frac_mic_unit', 'rec_eff_percent', 'rec_eff_target_name',
       'rec_eff_spike_matrix', 'rec_eff_spike_conc', 'date_updated'],
      dtype='object')


In [8]:
# Asegúrate de tener df_raw ya cargado
# df_raw = load_cdc_wastewater_full()  # o vía API

# Parseo de fecha por si acaso
df_raw["sample_collect_date"] = pd.to_datetime(
    df_raw["sample_collect_date"], errors="coerce"
)

# Nos quedamos solo con filas que tienen target numérico
TARGET_COL = "pcr_target_flowpop_lin"
DATE_COL = "sample_collect_date"

df_target = df_raw.dropna(subset=[DATE_COL, TARGET_COL]).copy()
print("Shape con target no nulo:", df_target.shape)

# --- Resumen por sewershed_id ---
sewershed_summary = (
    df_target
    .groupby("sewershed_id")[DATE_COL]
    .agg(["min", "max", "count"])
    .assign(span_days=lambda d: (d["max"] - d["min"]).dt.days)
    .sort_values("count", ascending=False)
)

print("TOP 20 sewersheds con más muestras:")
display(sewershed_summary.head(20))

# --- Resumen por estado / jurisdicción ---
state_summary = (
    df_target
    .groupby("wwtp_jurisdiction")[DATE_COL]
    .agg(["min", "max", "count"])
    .assign(span_days=lambda d: (d["max"] - d["min"]).dt.days)
    
    
    
    .sort_values("count", ascending=False)
)

print("TOP 20 estados con más muestras:")
display(state_summary.head(20))


Shape con target no nulo: (408473, 35)
TOP 20 sewersheds con más muestras:


,min,max,count,span_days
sewershed_id,,,,
752,2020-01-14 00:00:00+00:00,2025-09-10 00:00:00+00:00,1856,2066
743,2020-01-14 00:00:00+00:00,2025-09-10 00:00:00+00:00,1852,2066
2113,2020-09-14 00:00:00+00:00,2025-09-22 00:00:00+00:00,1791,1834
1181,2021-09-14 00:00:00+00:00,2025-09-22 00:00:00+00:00,1622,1469
954,2021-07-22 00:00:00+00:00,2025-09-13 00:00:00+00:00,1360,1514
966,2021-07-21 00:00:00+00:00,2025-09-13 00:00:00+00:00,1298,1515
2093,2020-11-01 00:00:00+00:00,2025-09-15 00:00:00+00:00,1286,1779
888,2022-01-12 00:00:00+00:00,2025-09-14 00:00:00+00:00,1227,1341
845,2021-07-22 00:00:00+00:00,2025-09-09 00:00:00+00:00,1205,1510


TOP 20 estados con más muestras:


,min,max,count,span_days
wwtp_jurisdiction,,,,
ny,2020-08-31 00:00:00+00:00,2025-09-17 00:00:00+00:00,40559,1843
oh,2020-07-19 00:00:00+00:00,2025-09-22 00:00:00+00:00,30094,1891
il,2021-11-01 00:00:00+00:00,2025-09-18 00:00:00+00:00,25773,1417
mi,2021-05-30 00:00:00+00:00,2025-09-15 00:00:00+00:00,25724,1569
wi,2020-08-23 00:00:00+00:00,2025-09-22 00:00:00+00:00,21710,1856
ca,2020-04-20 00:00:00+00:00,2025-09-23 00:00:00+00:00,20050,1982
co,2020-08-02 00:00:00+00:00,2025-09-17 00:00:00+00:00,17601,1872
nc,2020-12-13 00:00:00+00:00,2025-09-23 00:00:00+00:00,15222,1745
mo,2020-07-05 00:00:00+00:00,2025-09-23 00:00:00+00:00,14656,1906


## Exploración y limpieza básica de la serie seleccionada


In [9]:
# === OPCIÓN B: SERIE POR ESTADO (AGREGADO) ===

STATE = "ny"  # ej. "CA", "TX", etc.

DATE_COL = "sample_collect_date"
TARGET_COL = "pcr_target_flowpop_lin"

df_state = (
    df_target
    .query("wwtp_jurisdiction == @STATE")
    [[DATE_COL, TARGET_COL]]
    .copy()
)

df_state[DATE_COL] = pd.to_datetime(df_state[DATE_COL], errors="coerce")
df_state = df_state.dropna(subset=[DATE_COL, TARGET_COL])

# Agregamos por fecha (ejemplo: mediana entre sewersheds ese día)
df_state = (
    df_state
    .groupby(DATE_COL, as_index=False)[TARGET_COL]
    .median()
    .sort_values(DATE_COL)
)

df_site = df_state.rename(columns={DATE_COL: "date", TARGET_COL: "target"})

print("Estado:", STATE)
print("Rango de fechas:", df_site["date"].min(), "→", df_site["date"].max())
print("Número de filas:", len(df_site))
display(df_site.head())

fig = plot_time_series(
    df_site["date"],
    df_site["target"],
    title=f"Serie – estado {STATE} (mediana sewersheds)",
    ylabel="Carga viral normalizada (flow/pop)"
)
fig.show()


Estado: ny
Rango de fechas: 2020-08-31 00:00:00+00:00 → 2025-09-17 00:00:00+00:00
Número de filas: 1366


,date,target
0,2020-08-31 00:00:00+00:00,2.832473e+06
1,2020-09-02 00:00:00+00:00,5.965088e+02
2,2020-09-04 00:00:00+00:00,2.355929e+04
3,2020-09-06 00:00:00+00:00,6.226106e+02
4,2020-09-08 00:00:00+00:00,5.880741e+06


In [10]:
# TODO: ejecuta esta celda cuando df_site exista y tenga columnas ['date', 'target']

df_site = df_site.drop_duplicates(subset=["date"]).reset_index(drop=True)
df_site = df_site.sort_values("date")

print("Rango de fechas:", df_site["date"].min(), "→", df_site["date"].max())
print("Número de filas (tras limpieza):", len(df_site))
display(df_site.head())
display(df_site.describe())

# Tratamiento simple de NA (puedes mejorarlo si es necesario)
df_site["target"] = df_site["target"].interpolate().fillna(method="bfill").fillna(method="ffill")

# Gráfica rápida
plot_time_series(df_site["date"], df_site["target"], title="Serie completa – carga viral", ylabel="target")
fig.show()


Rango de fechas: 2020-08-31 00:00:00+00:00 → 2025-09-17 00:00:00+00:00
Número de filas (tras limpieza): 1366


,date,target
0,2020-08-31 00:00:00+00:00,2.832473e+06
1,2020-09-02 00:00:00+00:00,5.965088e+02
2,2020-09-04 00:00:00+00:00,2.355929e+04
3,2020-09-06 00:00:00+00:00,6.226106e+02
4,2020-09-08 00:00:00+00:00,5.880741e+06


,target
count,1.366000e+03
mean,2.912826e+07
std,9.036225e+07
min,4.811213e+02
25%,6.187153e+05
50%,9.393663e+06
75%,2.312675e+07
max,1.854606e+09


C:\Users\esteb\AppData\Local\Temp\ipykernel_3172\1573644186.py:12: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



## Preparación del dataset para modelado (ventanas deslizantes)


In [ ]:
# Configuración de ventana y horizonte de predicción

# TODO: ajusta según tu problema
WINDOW_SIZE = 30   # días/semanas hacia atrás que ve el modelo
HORIZON = 1        # predicción one-step-ahead
TRAIN_FRAC = 0.7
VAL_FRAC = 0.15

# TODO: ejecuta cuando df_site exista
# series = df_site.set_index("date")["target"].astype(float)
# values = series.values
# dates = series.index

# X_all, y_all = create_sliding_windows(values, window_size=WINDOW_SIZE, horizon=HORIZON)
# print("X_all shape:", X_all.shape)
# print("y_all shape:", y_all.shape)

# (X_train, y_train), (X_val, y_val), (X_test, y_test) = time_series_train_val_test_split(
#     X_all, y_all, train_frac=TRAIN_FRAC, val_frac=VAL_FRAC
# )
# print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# # Fechas correspondientes al target de cada ventana
# target_dates = dates[WINDOW_SIZE:WINDOW_SIZE + len(y_all)]
# train_dates = target_dates[: len(y_train)]
# val_dates = target_dates[len(y_train): len(y_train) + len(y_val)]
# test_dates = target_dates[len(y_train) + len(y_val):]


In [ ]:
# Escalamiento opcional (útil para redes neuronales)

# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)

# # TODO: decide si vas a trabajar con las versiones escaladas o sin escalar.


## Modelos de pronóstico

Vamos a definir primero un **baseline ingenuo**, y luego uno o más **modelos no lineales**.


In [ ]:
# === Baseline ingenuo (último valor) ===

# TODO: ejecuta cuando tengas X_train, y_train, etc.

# baseline = NaiveLastValueModel().fit(X_train, y_train)

# y_train_pred = baseline.predict(X_train).reshape(-1)
# y_val_pred = baseline.predict(X_val).reshape(-1)
# y_test_pred = baseline.predict(X_test).reshape(-1)

# y_train_true = y_train.reshape(-1)
# y_val_true = y_val.reshape(-1)
# y_test_true = y_test.reshape(-1)

# metrics_train = regression_metrics(y_train_true, y_train_pred)
# metrics_val = regression_metrics(y_val_true, y_val_pred)
# metrics_test = regression_metrics(y_test_true, y_test_pred)

# print("Baseline – métricas train:", metrics_train)
# print("Baseline – métricas val:", metrics_val)
# print("Baseline – métricas test:", metrics_test)

# plot_test_vs_pred(test_dates, y_test_true, y_test_pred, title="Baseline – Test vs predicción")
# plt.show()


### Modelo 1 – (ejemplo) LSTM / CNN / MLP

Aquí puedes implementar tu primer modelo no lineal (LSTM, CNN 1D, MLP, etc.). Debes:

- Definir la arquitectura y comentar para qué sirve cada parte.
- Justificar hiperparámetros (número de capas, neuronas, tamaño de kernel, dropout, LR, etc.).
- Mostrar `model.summary()` o equivalente y comentarlo.


In [ ]:
# TODO: Implementa aquí tu Modelo 1 (ejemplo LSTM con Keras)

# from tensorflow import keras
# from tensorflow.keras import layers

# # Ejemplo: usar X_train_scaled con shape (n_samples, WINDOW_SIZE)
# X_train_lstm = X_train_scaled.reshape(-1, WINDOW_SIZE, 1)
# X_val_lstm = X_val_scaled.reshape(-1, WINDOW_SIZE, 1)
# X_test_lstm = X_test_scaled.reshape(-1, WINDOW_SIZE, 1)

# INPUT_SHAPE = (WINDOW_SIZE, 1)

# model_1 = keras.Sequential([
#     layers.Input(shape=INPUT_SHAPE),
#     layers.LSTM(64, return_sequences=False),
#     layers.Dense(32, activation="relu"),
#     layers.Dense(HORIZON)
# ])

# model_1.compile(
#     loss="mse",
#     optimizer=keras.optimizers.Adam(learning_rate=1e-3),
#     metrics=["mae"]
# )

# model_1.summary()

# # TODO: entrena el modelo con early stopping
# # history = model_1.fit(
# #     X_train_lstm, y_train,
# #     validation_data=(X_val_lstm, y_val),
# #     epochs=100,
# #     batch_size=32,
# #     callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
# # )


In [ ]:
# Evaluación del Modelo 1

# TODO: adapta esta celda a tu modelo (LSTM/CNN/etc.)

# # y_train_pred_1 = model_1.predict(X_train_lstm).reshape(-1)
# # y_val_pred_1 = model_1.predict(X_val_lstm).reshape(-1)
# # y_test_pred_1 = model_1.predict(X_test_lstm).reshape(-1)

# # y_train_true = y_train.reshape(-1)
# # y_val_true = y_val.reshape(-1)
# # y_test_true = y_test.reshape(-1)

# # metrics_train_1 = regression_metrics(y_train_true, y_train_pred_1)
# # metrics_val_1 = regression_metrics(y_val_true, y_val_pred_1)
# # metrics_test_1 = regression_metrics(y_test_true, y_test_pred_1)

# # print("Modelo 1 – métricas train:", metrics_train_1)
# # print("Modelo 1 – métricas val:", metrics_val_1)
# # print("Modelo 1 – métricas test:", metrics_test_1)

# # plot_test_vs_pred(test_dates, y_test_true, y_test_pred_1, title="Modelo 1 – Test vs predicción")
# # plt.show()


## Pronósticos futuros y fechas específicas


In [ ]:
# Pronóstico a N_FUTURE pasos usando tu modelo final

# TODO: elige tu modelo final (baseline, model_1, etc.)
# final_model = model_1  # ejemplo

# # Usamos la última ventana de toda la serie
# # last_window = values[-WINDOW_SIZE:]
# # N_FUTURE = 14
# # future_preds = recursive_forecast(final_model, last_window, n_future=N_FUTURE)

# # Fechas futuras (si la serie es diaria)
# # last_date = dates[-1]
# # future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=N_FUTURE, freq="D")

# # plot_future_forecast(dates, values, future_dates, future_preds, title="Pronóstico futuro – carga viral")
# # plt.show()


In [ ]:
# Predicción para fechas específicas

# TODO: define fechas futuras concretas (en formato YYYY-MM-DD)
# specific_dates = [
#     "2025-01-01",
#     "2025-02-01",
#     "2025-03-01",
# ]

# # preds_specific = forecast_for_specific_dates(
# #     final_model,
# #     series=series,
# #     future_dates=specific_dates,
# #     window_size=WINDOW_SIZE
# # )

# # preds_specific.to_frame(name="forecast")


## Conclusiones y hallazgos

En esta sección deberás sintetizar:

- Qué aprendiste de la serie de tiempo (tendencias, cambios, ruidos raros).
- Qué modelo funcionó mejor y por qué.
- En qué condiciones falla tu modelo (picos, cambios de régimen, datos faltantes, etc.).
- Qué tan útil es el pronóstico para la toma de decisiones en el contexto de vigilancia en aguas residuales.
- Qué harías diferente con más tiempo (más features, tuning más fino, otros modelos, etc.).

Aquí también puedes dejar apuntes para tu README y tus diapositivas finales.
